# Session 4 - Topic 1 (Use Cases)
ASCII‑only examples with buffer protocol, memoryview, and safe release patterns.


### Case 1: Reverse in-place with memoryview

In [1]:
import array, struct, os, mmap, tempfile, sys

# Case 1: Reverse in-place with memoryview
def reverse_inplace(data):
    mv = memoryview(data)
    left, right = 0, len(mv) - 1
    while left < right:
        mv[left], mv[right] = mv[right], mv[left]
        left += 1
        right -= 1
    mv.release()

ba = bytearray(b"abcdefghijklmnopqrstuvwxyz")
reverse_inplace(ba)
print("Reverse in-place :", ba[:10], "...")

Reverse in-place : bytearray(b'zyxwvutsrq') ...


### Case 2: Binary header parse

In [3]:
import os
import struct

# Case 2: Binary Header Parse

# Step 1: Create a bytearray of 16 random bytes (could represent a binary file header)
header = bytearray(os.urandom(16))

# Step 2: Create a memoryview of the header to allow zero-copy access to its raw bytes
mv_head = memoryview(header)

# Step 3: Unpack the first 8 bytes of the header as:
#   - 1 x Big-endian unsigned int (4 bytes) -> 'magic'
#   - 2 x Big-endian unsigned short (2 bytes each) -> 'major', 'minor'
# Using the format string ">IHH" where:
#   '>' means big-endian
#   'I' means 4-byte unsigned int
#   'H' means 2-byte unsigned short
magic, major, minor = struct.unpack_from(">IHH", mv_head, 0)

# Step 4: Release the memoryview (good practice when working with low-level buffers)
mv_head.release()

# Step 5: Print the parsed values from the binary header
print("Parsed header magic:", magic)
print("Major version:", major)
print("Minor version:", minor)

Parsed header magic: 435235094
Major version: 1472
Minor version: 6596


### Case 3: mmap editing without copy

In [5]:
import mmap
import tempfile

# Case 3: mmap editing without copy

# Step 1: Create a temporary file and write some initial data to it
tmp = tempfile.TemporaryFile()
tmp.write(b"abcdefghij")  # Write 10 bytes of sample data
tmp.flush()               # Make sure data is written to disk

# Step 2: Memory-map the file so we can access and modify it directly in memory
# fileno() gives the OS-level file descriptor
# 0 as size means map the entire file
# access=mmap.ACCESS_WRITE allows reading and writing
mm = mmap.mmap(tmp.fileno(), 0, access=mmap.ACCESS_WRITE)

# Step 3: Get a memoryview of the mmap object
# This lets us work with the memory buffer directly, zero-copy
mv_map = memoryview(mm)

# Step 4: Modify the first 4 bytes of the memory-mapped file
# This changes the file on disk without needing to call write()
mv_map[0:4] = b"ZZZZ"

# Step 5: Print the updated content from the mmap view
print("After mmap edit  :", mm[:10])  # Should show b'ZZZZefghij'

# Step 6: Release the memoryview (good practice)
mv_map.release()

# Step 7: Close the mmap object to unmap the file from memory
mm.close()

# Step 8: Close the temporary file
tmp.close()

After mmap edit  : b'ZZZZefghij'


In [7]:
# Case 5: Packet assembly for TCP/IP or custom binary protocols

def build_packet(parts):
    """
    Builds a single binary packet from multiple byte-like parts.

    Args:
        parts (list of bytes/bytearray/memoryview): Components to assemble into one buffer

    Returns:
        bytearray: A single contiguous buffer containing all parts
    """
    # Step 1: Pre-allocate a bytearray of the exact size needed
    total_size = sum(len(p) for p in parts)
    buf = bytearray(total_size)

    # Step 2: Create a memoryview of the buffer for efficient slicing without copying
    mv = memoryview(buf)

    # Step 3: Copy each part into the correct position in the buffer
    offset = 0
    for part in parts:
        part_length = len(part)
        mv[offset:offset + part_length] = part  # No copy of the whole buffer
        offset += part_length  # Move to next writing position

    # Step 4: Release the memoryview (good practice, especially in C extensions)
    mv.release()

    # Step 5: Return the assembled packet
    return buf


# Example: Assemble a TCP-like packet with header and payload
pkt = build_packet([b"HEAD", b"DATA", b"TAIL"])
print("Assembled packet :", pkt)

Assembled packet : bytearray(b'HEADDATATAIL')
